In [1]:
from pyspark.sql import SparkSession

In [2]:
warehouse = "/user/team20/project/hive/warehouse"
team = "team20"

spark = SparkSession.builder\
        .appName("{} - spark ML".format(team))\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

In [3]:
spark.sql("SHOW DATABASES").show()
spark.sql("USE team20_projectdb").show()
spark.sql("SHOW TABLES").show()
spark.sql("SELECT * FROM anime_part_buck LIMIT 10").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|             root_db|
|     team0_projectdb|
|team12_hive_proje...|
|    team13_projectdb|
|    team14_projectdb|
|    team15_projectdb|
|    team16_projectdb|
|    team18_projectdb|
|    team19_projectdb|
|     team1_projectdb|
|    team20_projectdb|
|    team21_projectdb|
|    team22_projectdb|
|    team23_projectdb|
|    team25_projectdb|
|    team26_projectdb|
|    team27_projectdb|
|    team28_projectdb|
|    team29_projectdb|
+--------------------+
only showing top 20 rows

++
||
++
++

+----------------+---------------+-----------+
|       namespace|      tableName|isTemporary|
+----------------+---------------+-----------+
|team20_projectdb|anime_part_buck|      false|
|team20_projectdb|     q1_results|      false|
|team20_projectdb|     q2_results|      false|
|team20_projectdb|     q3_results|      false|
|team20_projectdb|     q4_results|      false|
|team20_projectdb|     q5_results

In [4]:
print(spark.catalog.listTables("team20_projectdb"))

[Table(name='anime_part_buck', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='q1_results', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='q2_results', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='q3_results', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='q4_results', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='q5_results', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='users_details', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='users_scores', database='team20_projectdb', description=None, tableType='EXTERNAL', isTemporary=False)]


In [5]:
spark.sql("SELECT count(*) FROM users_scores").show()


+--------+
|count(1)|
+--------+
|23796586|
+--------+



In [6]:
# data = spark.sql("SELECT user_id, anime_id, rating FROM users_scores")
data = spark.sql("SELECT user_id, anime_id, rating FROM users_scores ORDER BY user_id LIMIT 5000")

data.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      1|      21|     9|
|      1|      48|     7|
|      1|     320|     5|
|      1|      49|     8|
|      1|     304|     8|
|      1|     306|     8|
|      1|      53|     7|
|      1|      47|     5|
|      1|     591|     6|
|      1|      54|     7|
|      1|      55|     5|
|      1|      56|     6|
|      1|      57|     9|
|      1|     368|     5|
|      1|      68|     7|
|      1|     889|     9|
|      1|    1519|     7|
|      1|      58|     8|
|      1|    1222|     7|
|      1|     458|     4|
+-------+--------+------+
only showing top 20 rows



In [7]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer

# inspired by https://medium.com/@brunoborges_38708/recommender-system-using-als-in-pyspark-10329e1d1ee1

user_window = Window.partitionBy("user_id").orderBy(F.col("anime_id").desc())

df_rec_filtered = data.withColumn("num_items", F.expr("count(*) over (partition by user_id)"))
df_rec_filtered = df_rec_filtered.filter(F.col("num_items")>=5)


In [8]:
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer



# For example, 30% of items will be masked
percent_items_to_mask = 0.3

# Determine the number of items to mask for each user
df_rec_final = df_rec_filtered.withColumn("num_items_to_mask", (F.col("num_items") * percent_items_to_mask).cast("int"))

# Masks items for each user
# _tmp_window = Window.partitionBy("user_id").orderBy(F.rand(seed))
df_rec_final = df_rec_final.withColumn("item_rank", F.rank().over(user_window))

# Create a StringIndexer model to index the user ID column
# indexer_user = StringIndexer(inputCol='user_id', outputCol='userIndex').setHandleInvalid("keep")
# indexer_item = StringIndexer(inputCol='anime_id', outputCol='itemIndex').setHandleInvalid("keep")

# # Fit the indexer model to the data and transform the DataFrame
# df_rec_final = indexer_user.fit(df_rec_final).transform(df_rec_final)
# df_rec_final = indexer_item.fit(df_rec_final).transform(df_rec_final)

# Convert the userIndex column to integer type
# df_rec_final = df_rec_final.withColumn('userIndex', df_rec_final['userIndex'].cast('integer')) \
#     .withColumn('itemIndex', df_rec_final['itemIndex'].cast('integer'))

# Filter train and test DataFrames
train_df_rec = df_rec_final.filter(F.col("item_rank") > F.col("num_items_to_mask"))
test_df_rec = df_rec_final.filter(F.col("item_rank") <= F.col("num_items_to_mask"))
print(train_df_rec.count(), test_df_rec.count())

3514 1483


In [9]:
train_df_rec.write.json("/user/team20/project/data/train.json")
test_df_rec.write.json("/user/team20/project/data/test.json")

In [10]:
top_n_user = train_df_rec.groupBy("user_id").agg(F.count("rating").alias("n_watched")).orderBy(F.desc("n_watched")).limit(1000)
train_df_rec = train_df_rec.join(top_n_user, on="user_id", how="inner")
test_df_rec = test_df_rec.join(top_n_user, on="user_id", how="inner")

print(train_df_rec.count(), test_df_rec.count())

3514 1483


In [11]:
top_n_user.show()

+-------+---------+
|user_id|n_watched|
+-------+---------+
|    119|      285|
|    162|      208|
|      4|      199|
|     23|      198|
|      1|      184|
|    185|      176|
|    138|      159|
|    208|      159|
|    133|      147|
|    157|      145|
|     66|      139|
|     70|      126|
|     82|      116|
|    222|      112|
|    212|      108|
|     47|      101|
|     95|       89|
|    112|       89|
|    120|       88|
|    143|       82|
+-------+---------+
only showing top 20 rows



In [12]:
import pyspark

# def does_contain_same(train: pyspark.sql.dataframe.DataFrame, test: pyspark.sql.dataframe.DataFrame, column: str) -> bool:
#     tr_dist = train.select(column).distinct().orderBy(F.col(column).asc())
#     test_dist = test.select(column).distinct().orderBy(F.col(column).asc())
#     return tr_dist.collect() == test_dist.collect()

# does_contain_same(train_df_rec, test_df_rec, "user_id")


In [13]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


als = ALS(userCol='user_id', itemCol='anime_id', ratingCol='rating',
          coldStartStrategy='drop')

# param_grid = ParamGridBuilder()\
#              .addGrid(als.rank, [1, 20, 30])\
#              .addGrid(als.maxIter, [10, 20])\
#              .addGrid(als.regParam, [.05, .15])\
#              .build()

param_grid = ParamGridBuilder()\
             .addGrid(als.rank, [5, 20])\
             .addGrid(als.maxIter, [10, 20])\
             .addGrid(als.regParam, [.05, .15])\
             .build()

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

model = cv.fit(train_df_rec)
best_model = model.bestModel

print('rank: ', best_model.rank)
print('MaxIter: ', best_model._java_obj.parent().getMaxIter())
print('RegParam: ', best_model._java_obj.parent().getRegParam())

rank:  30
MaxIter:  20
RegParam:  0.15


In [14]:
from pyspark.ml.recommendation import ALS

als = ALS(rank=best_model.rank, maxIter=best_model._java_obj.parent().getMaxIter(), regParam=best_model._java_obj.parent().getRegParam(), userCol="user_id", itemCol="anime_id")
model = als.fit(train_df_rec)

In [15]:
predictions = model.recommendForAllUsers(10)
predictions.select("recommendations").show()

/usr/local/lib/python3.6/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+
|     recommendations|
+--------------------+
|[{44, 9.748807}, ...|
|[{114, 9.630049},...|
|[{21, 10.22297}, ...|
|[{1142, 9.574842}...|
|[{2593, 9.854924}...|
|[{122, 9.226507},...|
|[{75, 9.916822}, ...|
|[{44, 9.578612}, ...|
|[{1889, 9.586806}...|
|[{777, 9.97529}, ...|
|[{164, 8.844166},...|
|[{121, 10.919873}...|
|[{92, 9.866216}, ...|
|[{1142, 9.668254}...|
|[{121, 11.036204}...|
|[{1689, 9.72442},...|
|[{467, 9.658599},...|
|[{21, 9.651628}, ...|
|[{801, 9.856971},...|
|[{302, 9.871355},...|
+--------------------+
only showing top 20 rows



In [16]:
NUMBER2RECOMMEND = best_model.rank

most_rated_anime = df_rec_filtered.groupBy("anime_id")\
                    .agg(F.mean("rating").alias("avg_rating"), F.count("rating").alias("count"))\
                    .where("avg_rating > 8")\
                    .orderBy(F.desc("count")).limit(NUMBER2RECOMMEND)
most_rated_anime = most_rated_anime.select("anime_id")

In [17]:
predictions = predictions.withColumn("recommendations", F.transform(F.col('recommendations'), lambda x: x.anime_id))
predictions.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|     20|[44, 1827, 121, 1...|
|    120|[114, 139, 138, 1...|
|    130|[21, 136, 44, 121...|
|     80|[1142, 16, 121, 1...|
|     70|[2593, 1072, 67, ...|
|     91|[122, 121, 19, 21...|
|    111|[75, 121, 30, 245...|
|      1|[44, 199, 136, 26...|
|     71|[1889, 431, 160, ...|
|    222|[777, 282, 264, 3...|
|    212|[164, 593, 75, 44...|
|    112|[121, 1689, 1889,...|
|    162|[92, 2457, 1142, ...|
|     82|[1142, 16, 489, 2...|
|     53|[121, 16, 1889, 1...|
|    133|[1689, 953, 121, ...|
|    223|[467, 164, 199, 4...|
|    163|[21, 122, 269, 26...|
|     23|[801, 467, 777, 2...|
|    143|[302, 283, 121, 7...|
+-------+--------------------+
only showing top 20 rows



In [18]:
test_rec_list = test_df_rec.select("user_id", "anime_id", "rating").orderBy("user_id", F.desc("rating")).groupBy("user_id").agg(F.collect_list("anime_id").alias("gt"))
test_rec_list.show()

+-------+--------------------+
|user_id|                  gt|
+-------+--------------------+
|      1|[1698, 849, 32281...|
|      4|[31043, 38003, 33...|
|      9|[317, 304, 392, 3...|
|     20|[11061, 9253, 364...|
|     23|[22535, 22199, 18...|
|     47|[6351, 4059, 2963...|
|     53|[48411, 42847, 41...|
|     66|[5680, 4224, 3958...|
|     70|[8074, 5258, 5114...|
|     71|[5258, 2890, 1535...|
|     80|[4224, 4483, 4477...|
|     82|[36296, 35120, 37...|
|     83|[16774, 30240, 20...|
|     91|[995, 4053, 2752,...|
|     95|[28771, 22507, 22...|
|    108|[4177, 3572, 3588...|
|    111|[529, 433, 390, 2...|
|    112|[38524, 38000, 35...|
|    116|[323, 345, 269, 2...|
|    119|[20785, 19815, 19...|
+-------+--------------------+
only showing top 20 rows



In [19]:
predictions.count(), test_rec_list.count()

(36, 36)

In [20]:
recomendations = test_rec_list.join(predictions, on="user_id", how="inner")
recomendations.show()

+-------+--------------------+--------------------+
|user_id|                  gt|     recommendations|
+-------+--------------------+--------------------+
|      1|[1698, 849, 32281...|[44, 199, 136, 26...|
|      4|[31043, 38003, 33...|[3901, 3784, 4725...|
|      9|[317, 304, 392, 3...|[44, 226, 72, 139...|
|     20|[11061, 9253, 364...|[44, 1827, 121, 1...|
|     23|[22535, 22199, 18...|[801, 467, 777, 2...|
|     47|[6351, 4059, 2963...|[1689, 1530, 256,...|
|     53|[48411, 42847, 41...|[121, 16, 1889, 1...|
|     66|[5680, 4224, 3958...|[75, 106, 98, 232...|
|     70|[8074, 5258, 5114...|[2593, 1072, 67, ...|
|     71|[5258, 2890, 1535...|[1889, 431, 160, ...|
|     80|[4224, 4483, 4477...|[1142, 16, 121, 1...|
|     82|[36296, 35120, 37...|[1142, 16, 489, 2...|
|     83|[16774, 30240, 20...|[121, 1142, 3784,...|
|     91|[995, 4053, 2752,...|[122, 121, 19, 21...|
|     95|[28771, 22507, 22...|[1894, 1889, 413,...|
|    108|[4177, 3572, 3588...|[306, 3358, 656, ...|
|    111|[52

In [21]:
recomendations.write.parquet("/user/team20/project/output/model1_predictions")

In [22]:
model.save("/user/team20/project/models/model1.parquet")

In [23]:
from pyspark.mllib.evaluation import RankingMetrics

metrics = RankingMetrics(recomendations.select("gt", "recommendations").rdd)

In [24]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType


schema = StructType([
    StructField("model", StringType(), True),
    StructField("precision@10", FloatType(), True),
    StructField("recall@10", FloatType(), True),
    StructField("ndcg@10", FloatType(), True),
    StructField("precision@5", FloatType(), True),
    StructField("recall@5", FloatType(), True),
    StructField("ndcg@5", FloatType(), True)
])

# Create a list of tuples with sample data
data = [(
    "avg_synopsis_emb",
    metrics.precisionAt(10),
    metrics.recallAt(10),
    metrics.ndcgAt(10),
    metrics.precisionAt(5),
    metrics.recallAt(5),
    metrics.ndcgAt(5)
)]

df = spark.createDataFrame(data, schema)
df.write.parquet("/user/team20/project/output/evaluation", mode="append")

In [32]:
with open("/home/team20/team20/bigdata-final-project-iu-2024.git/output/evaluation.csv", "w") as f:
    f.write("model,precision@10,recall@10,ndcg@10,precision@5,recall@5,ndcg@5\n")
    f.write("ALS,")
    for v in [10, 5]:
        f.write(",".join(map(str, [metrics.precisionAt(v), metrics.recallAt(v), metrics.ndcgAt(v)])))
        if v == 10:
            f.write(",")
    f.write("\n")
    

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 55272)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/local/lib/python3.6/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.6/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/lib/python3.6/site-packages/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File 

In [26]:
# most_rated_anime_list = most_rated_anime.rdd.map(lambda x: x.anime_id).collect()

# most_rated_anime_df = test_rec_list.select("user_id", "gt")
# most_rated_anime_df = most_rated_anime_df.withColumn("recommendations", F.array([F.lit(x) for x in most_rated_anime_list]))
# most_rated_anime_df.show()

In [27]:
# most_popular_metrics = RankingMetrics(most_rated_anime_df.select("gt", "recommendations").rdd)
# 

In [28]:
# most_popular_metrics.ndcgAt(50)

In [29]:
# most_popular_metrics.precisionAt(50)